In [66]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 300)
import math
import warnings
warnings.filterwarnings('ignore')
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

%matplotlib inline

In [67]:
#Reading the file
data = pd.read_csv('files_for_lab/customer_churn.csv')
data.shape
data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [68]:
data2 = data.copy()
data2.columns = [e.lower().replace(' ', '_') for e in data2.columns]
data2 = data2.drop_duplicates()

In [69]:
data2.head()

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [70]:
data3 = data2[['tenure','seniorcitizen','monthlycharges','churn']]
data3.head()

,tenure,seniorcitizen,monthlycharges,churn
0,1,0,29.85,No
1,34,0,56.95,No
2,2,0,53.85,Yes
3,45,0,42.30,No
4,2,0,70.70,Yes


In [71]:
x_num = data3.select_dtypes(include=np.number).replace(np.nan, 0).fillna(0)
x_num = data3.drop(['churn','seniorcitizen'], axis = 1)
x_num.isna().sum()

tenure            0
monthlycharges    0
dtype: int64

In [78]:
x_normalized = Normalizer().fit_transform(x_num)
x_standardized = StandardScaler().fit_transform(x_num)

In [79]:
x_num2 = pd.DataFrame(x_normalized)
x_num2.shape

(7043, 2)

In [83]:
x_num2 = pd.concat([x_num2, data3['seniorcitizen']], axis = 1)
x_num2.shape

(7043, 3)

In [84]:
x = x_num2
y = data3['churn']

In [85]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state=100)

In [86]:
from sklearn.linear_model import LogisticRegression
classification = LogisticRegression(random_state=0, solver='lbfgs',
                  multi_class='multinomial').fit(x_train, y_train)

In [87]:
classification.score(x_test, y_test)

0.7685754850922859

In [97]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()
x_sm, y_sm = smote.fit_sample(x, y)
y_sm.value_counts()

No     5174
Yes    5174
Name: churn, dtype: int64

In [102]:
x_train, x_test, y_train, y_test = train_test_split(x_sm, y_sm, test_size = 0.3, random_state=100)
from sklearn.linear_model import LogisticRegression
classification = LogisticRegression(random_state=0, solver='lbfgs',
                  multi_class='multinomial').fit(x_train, y_train)
classification.score(x_test, y_test)

0.7513687600644122

In [107]:
from imblearn.under_sampling import TomekLinks
tl = TomekLinks('majority')
x_tl, y_tl = tl.fit_sample(x, y)
y_tl.value_counts()

No     4654
Yes    1869
Name: churn, dtype: int64

In [108]:
x_train, x_test, y_train, y_test = train_test_split(x_tl, y_tl, test_size = 0.3, random_state=100)
from sklearn.linear_model import LogisticRegression
classification = LogisticRegression(random_state=0, solver='lbfgs',
                  multi_class='multinomial').fit(x_train, y_train)
classification.score(x_test, y_test)

0.7731221257026061